<a href="https://colab.research.google.com/github/NeuroDesk/example-notebooks/blob/main/diffusion_imaging/mrtrix3tissue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>   </a>

# MRtrix3tissue

Author: Thuy Dao

Citation: 3tissue.github.io/about.html

## Setup Neurodesk

In [1]:
%%capture
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  os.environ["LD_PRELOAD"] = "";
  os.environ["APPTAINER_BINDPATH"] = "/content,/tmp,/cvmfs"
  os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
  os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

  !curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
  !chmod +x googlecolab_setup.sh
  !./googlecolab_setup.sh

  os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

In [2]:
# Output CPU information:
!cat /proc/cpuinfo | grep 'vendor' | uniq
!cat /proc/cpuinfo | grep 'model name' | uniq

vendor_id	: AuthenticAMD
model name	: AMD EPYC 7742 64-Core Processor


In [1]:
import subprocess
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import tempfile

## Introduction

In [1]:
# load fmriprep
import lmod
import os
# await lmod.load('mrtrix3/3.0.4')
await lmod.load('mrtrix3tissue/5.2.8')
await lmod.list()

['mrtrix3/3.0.4']

## Data preparation

In [4]:
# download data
!datalad install https://github.com/OpenNeuroDatasets/ds002080.git
!cd ds002080 && datalad get sub-CON02

Cloning:   0%|                             | 0.00/2.00 [00:00<?, ? candidates/s]
Enumerating: 0.00 Objects [00:00, ? Objects/s]
                                              
Counting:   0%|                              | 0.00/20.6k [00:00<?, ? Objects/s]
                                                                                
Compressing:   0%|                           | 0.00/12.7k [00:00<?, ? Objects/s]
                                                                                
Receiving:   0%|                             | 0.00/22.6k [00:00<?, ? Objects/s]
Receiving:   9%|█▊                  | 2.03k/22.6k [00:00<00:01, 18.3k Objects/s]
Receiving:  31%|██████▏             | 6.99k/22.6k [00:00<00:00, 35.3k Objects/s]
Receiving:  49%|█████████▊          | 11.1k/22.6k [00:00<00:00, 37.2k Objects/s]
Receiving:  83%|████████████████▌   | 18.7k/22.6k [00:00<00:00, 52.1k Objects/s]
                                                                                
Resolving:   0%

### Convert to .mif file
Once data is downloaded, it is recommended to import and store it as .mif file(s), the so-called “MRtrix Image Format”.

`mrconvert` is the tool to convert between all sorts of image formats (or extract parts of images or change properties) and enables you to read from e.g. DICOM folders or NIfTI (.nii) images and store an image as a .mif file.

If your diffusion MRI data comes as a NIfTI (.nii) image instead, the gradient orientations and b-values will typically be stored in 2 separate files (a “bvecs” and “bvals” file).

In [3]:
!mrconvert ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.nii.gz AP.mif -fslgrad ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bvec ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bval
!mrconvert ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-PA_dwi.nii.gz PA.mif -fslgrad ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-PA_dwi.bvec ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-PA_dwi.bval

mrconvert: [100%] uncompressing image "ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.nii.gz"[0K
mrconvert: [100%] copying from "ds002080/s...es-postop_acq-AP_dwi.nii.gz" to "AP.mif"[0K


## Preprocessing
The following are common preprocessing steps done with MRtrix. If you have used the software package FSL to analyze diffusion data, note that some of the FSL commands - such as eddy and topup - are used in some of the MRtrix libraries. 

### Denoising and Gibbs ringing removal (“unringing”)


In [4]:
!dwidenoise AP.mif dwi_denoised.mif -noise noise.mif
!mrdegibbs dwi_denoised.mif dwi_denoised_unringed.mif

dwidenoise: [100%] preloading data for "AP.mif"[0K
dwidenoise: [100%] running MP-PCA denoising[0K
mrdegibbs: [100%] performing Gibbs ringing removal[0K


### Motion and distortion correction

Motion and distortion correction are performed by the `dwifslpreproc` command, most of the heavy lifting is done automatically by FSL’s `topup` and `eddy` tools.

If you have also scanned a pair of reverse phase-encoded b=0 images, you can correct for susceptibility-induced EPI distortions by concatenating the reverse phase-encoded b=0 images using `mrcat`.

In [4]:
# !mrconvert ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-PA_dwi.nii.gz PA.mif
# !mrconvert PA.mif -fslgrad ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-PA_dwi.bvec ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-PA_dwi.bval - | mrmath - mean mean_b0_PA.mif -axis 3


mrconvert: [100%] uncompressing image "ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-PA_dwi.nii.gz"
mrconvert: [100%] copying from "ds002080/s...es-postop_acq-PA_dwi.nii.gz" to "PA.mif"


In [3]:
# !dwiextract dwi.mif - -fslgrad ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bvec ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bval -bzero | mrmath - mean mean_b0_AP.mif -axis 3
# !mrcat mean_b0_AP.mif mean_b0_PA.mif -axis 3 b0_pair.mif
!mrcat AP.mif PA.mif bzero_pair.mif -axis 3
!dwifslpreproc dwi_denoised_unringed.mif dwi_denoised_unringed_preproc.mif -pe_dir AP -rpe_pair -se_epi bzero_pair.mif  -eddy_options " --slm=linear --data_is_shelled"

mrcat: [100%] concatenating "AP.mif"
mrcat: [100%] concatenating "PA.mif"
dwifslpreproc: 
dwifslpreproc: Note that this script makes use of commands / algorithms that have relevant articles for citation; INCLUDING FROM EXTERNAL SOFTWARE PACKAGES. Please consult the help page (-help option) for more information.
dwifslpreproc: 
dwifslpreproc: Generated scratch directory: /data/books/diffusion_imaging/dwifslpreproc-tmp-H6QPID/
Command:  mrconvert /data/books/diffusion_imaging/dwi_denoised_unringed.mif /data/books/diffusion_imaging/dwifslpreproc-tmp-H6QPID/dwi.mif -json_export /data/books/diffusion_imaging/dwifslpreproc-tmp-H6QPID/dwi.json
Command:  mrconvert /data/books/diffusion_imaging/bzero_pair.mif /data/books/diffusion_imaging/dwifslpreproc-tmp-H6QPID/se_epi.mif
dwifslpreproc: Changing to scratch directory (/data/books/diffusion_imaging/dwifslpreproc-tmp-H6QPID/)
Command:  dirstat dwi.mif -output asym
dwifslpreproc: [WARNING] sampling of b=700 shell is moderately asymmetric; distort

In [8]:
# !dwifslpreproc dwi.mif dwi_preproc.mif -fslgrad ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bvec ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bval -pe_dir AP -rpe_pair -se_epi b0_pair.mif -eddy_options " --slm=linear --data_is_shelled"

dwifslpreproc: 
dwifslpreproc: Note that this script makes use of commands / algorithms that have relevant articles for citation; INCLUDING FROM EXTERNAL SOFTWARE PACKAGES. Please consult the help page (-help option) for more information.
dwifslpreproc: 
dwifslpreproc: Generated scratch directory: /data/books/diffusion_imaging/dwifslpreproc-tmp-K2AGQ4/
Command:  mrconvert /data/books/diffusion_imaging/dwi.mif /data/books/diffusion_imaging/dwifslpreproc-tmp-K2AGQ4/dwi.mif -fslgrad /data/books/diffusion_imaging/ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bvec /data/books/diffusion_imaging/ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bval -json_export /data/books/diffusion_imaging/dwifslpreproc-tmp-K2AGQ4/dwi.json
Command:  mrconvert /data/books/diffusion_imaging/b0_pair.mif /data/books/diffusion_imaging/dwifslpreproc-tmp-K2AGQ4/se_epi.mif
dwifslpreproc: Changing to scratch directory (/data/books/diffusion_imaging/dwifslpreproc-tmp-K2AGQ4/)
dwifs

####
When it has finished running, examine the output to see how eddy current correction and unwarping have changed the data; ideally, you should see more signal restored in regions such as the orbitofrontal cortex, which is particularly susceptible to signal dropout.

Let's display the newly preprocessed data, with the original diffusion data overlaid on top of it and colored in red. You should see a noticeable difference between the two images, especially in the frontal lobes of the brain near the eyes, which are most susceptible to eddy currents.

In [ ]:
# File paths for both images
mif_file_path1 = 'dwi_denoised_unringed_preproc.mif'  
mif_file_path2 = 'ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.nii.gz'          

# Convert both .mif files to temporary .nii.gz files with -force flag
with tempfile.NamedTemporaryFile(suffix=".nii.gz") as temp_file1, \
     tempfile.NamedTemporaryFile(suffix=".nii.gz") as temp_file2:
         
    # Convert main image
    result1 = subprocess.run(["mrconvert", mif_file_path1, temp_file1.name, "-force"], capture_output=True, text=True)
    if result1.returncode != 0:
        print(f"Error in mrconvert for preprocessed image: {result1.stderr}")
        exit()
    
    # Convert overlay image
    result2 = subprocess.run(["mrconvert", mif_file_path2, temp_file2.name, "-force"], capture_output=True, text=True)
    if result2.returncode != 0:
        print(f"Error in mrconvert for original image: {result2.stderr}")
        exit()
         
    # Load the converted images
    nii_image1 = nib.load(temp_file1.name)
    nii_image2 = nib.load(temp_file2.name)

    data1 = nii_image1.get_fdata()
    data2 = nii_image2.get_fdata()

    # Select middle slice 
    slice_index = data1.shape[2] // 3

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    # Plot preprocessed data
    axes[0].imshow(np.rot90(data1[:, :, slice_index, 0]), cmap="Greys_r", vmin=0, vmax=1000)
    axes[0].set_title("Preprocessed data")
    axes[0].axis("off")

    # Plot original diffusion data as overlay
    axes[1].imshow(np.rot90(data1[:, :, slice_index, 0]), cmap="Greys_r", vmin=0, vmax=1000) 
    axes[1].imshow(np.rot90(data2[:, :, slice_index, 0]), cmap="hot", alpha=0.4, vmin=0, vmax=1000) 
    axes[1].set_title("Preprocessed data with original diffusion data as overlay")
    axes[1].axis("off")
    plt.show()

In [ ]:
# !dwibiascorrect ants dwi_preproc.mif dwi_denoised_preproc_unbiased.mif -bias bias.mif
!dwibiascorrect ants dwi_denoised_unringed_preproc.mif dwi_denoised_unringed_preproc_unbiased.mif

In [ ]:
!dwi2mask dwi_denoised_unringed_preproc_unbiased.mif dwi_mask.mif

## 3-tissue response function estimation
A robust and fully automated unsupervised method to obtain 3-tissue response functions representing single-fibre white matter (WM), grey matter (GM) and CSF from the data itself, is available as the so-called dwi2response dhollander comman from `MRtrix3Tissue` package 

In [11]:
# await lmod.load('mrtrix3tissue/5.2.8')

In [ ]:
!dwi2response dhollander dwi_denoised_unringed_preproc_unbiased.mif response_wm.txt response_gm.txt response_csf.txt

In [ ]:
!mrgrid dwi_mask.mif regrid - -template dwi_denoised_unringed_preproc_unbiased_upsampled.mif -interp linear -datatype bit | maskfilter - median dwi_mask_upsampled.mif

In [5]:
# !dwi2mask dwi_denoised_preproc_unbiased_upsampled.mif dwi_mask_upsampled.mif

dwi2mask: [100%] preloading data for "dwi_denoised_preproc_unbiased_upsampled.mif"[0K
dwi2mask: [done] computing dwi brain mask[0K
dwi2mask: [ERROR] output file "dwi_mask_upsampled.mif" already exists (use -force option to force overwrite)
dwi2mask: [ERROR] error creating image "dwi_mask_upsampled.mif"


## 3-tissue CSD modelling
Your data is now ready for 3-tissue CSD modelling with the previously obtained 3-tissue response functions, which will result in modelling the diffusion MRI data using WM-like (FOD), GM-like and CSF-like compartments. There are 2 different methods (or algorithms) available to perform 3-tissue CSD. The choice between both depends on what (part of your) data you intend to perform 3-tissue CSD modelling for.

If you want to perform 3-tissue CSD modelling for multi-shell data, this can be achieved using the multi-shell multi-tissue CSD (MSMT-CSD) method or algorithm, as follows:

In [ ]:
!dwi2fod msmt_csd dwi_denoised_unringed_preproc_unbiased_upsampled.mif response_wm.txt wmfod.mif response_gm.txt gm.mif response_csf.txt csf.mif -mask dwi_mask_upsampled.mif

In [ ]:
!mtnormalise wmfod.mif wmfod_norm.mif gm.mif gm_norm.mif csf.mif csf_norm.mif -mask dwi_mask_upsampled.mif

 get CSF mask from csf spherical harmonic coefficients applying the first criteria (aka the csf.mif file):

In [ ]:
!mrconvert wmfod_norm.mif -coord 3 0 -axes 0,1,2 - | mrcalc csf_norm.mif gm_norm.mif - -add 5 -mult -gt csf_mask1.mif -datatype bit

get CSF mask from second criteria (note: 0.141047 might be half of max response function)

In [ ]:
!mrthreshold csf_norm.mif csf_mask2.mif -abs 0.141047

Getting the overlap of both masks:

In [ ]:
!mrcalc csf_mask1.mif csf_mask2.mif -mult csf_maskcombined.mif -datatype bit

In [ ]:
bvals = np.loadtxt("ds002080/sub-CON02/ses-postop/dwi/sub-CON02_ses-postop_acq-AP_dwi.bval")
bvals_filtered = np.where(bvals < 1000)

with open("bval_less_1000.txt", "w") as txt_file:
    txt_file.write(",".join((str(x) for x in bvals_filtered[0])))

In [ ]:
!mrconvert dwi_denoised_unringed_preproc_unbiased_upsampled.mif reduced.mif -coord 3 $(cat bval_less_1000.txt)

In [ ]:
!dwi2adc reduced.mif dwi_adc.mif

In [ ]:
!mrconvert dwi_adc.mif extracted_adc.mif -coord 3 1

In [ ]:
!mrcalc extracted_adc.mif csf_maskcombined.mif -mult masked_adc.mif

## Results

In [ ]:
!mrcalc 2256.74 4.39221 masked_adc.mif -divide -log -divide 273.15 -subtract masked_temperature.mif

In [ ]:
# File paths for both images
mif_file_path1 = 'dwi_denoised_unringed_preproc_unbiased_upsampled.mif'  
mif_file_path2 = 'masked_temperature.mif'          

# Convert both .mif files to temporary .nii.gz files with -force flag
with tempfile.NamedTemporaryFile(suffix=".nii.gz") as temp_file1, \
     tempfile.NamedTemporaryFile(suffix=".nii.gz") as temp_file2:
         
    # Convert main image
    result1 = subprocess.run(["mrconvert", mif_file_path1, temp_file1.name, "-force"], capture_output=True, text=True)
    if result1.returncode != 0:
        print(f"Error in mrconvert for preprocessed image: {result1.stderr}")
        exit()
    
    # Convert overlay image
    result2 = subprocess.run(["mrconvert", mif_file_path2, temp_file2.name, "-force"], capture_output=True, text=True)
    if result2.returncode != 0:
        print(f"Error in mrconvert for original image: {result2.stderr}")
        exit()
         
    # Load the converted images
    nii_image1 = nib.load(temp_file1.name)
    nii_image2 = nib.load(temp_file2.name)

    data1 = nii_image1.get_fdata()
    data2 = nii_image2.get_fdata()

    # Select middle slice 
    slice_index = data1.shape[2] // 3

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    # Plot preprocessed data
    axes[0].imshow(np.rot90(data1[:, :, slice_index, 0]), cmap="Greys_r", vmin=0, vmax=1000)
    axes[0].set_title("Preprocessed data")
    axes[0].axis("off")

    # Plot original diffusion data as overlay
    axes[1].imshow(np.rot90(data1[:, :, slice_index, 0]), cmap="Greys_r", vmin=0, vmax=1000) 
    axes[1].imshow(np.rot90(data2[:, :, slice_index, 0]), cmap="hot", alpha=0.4, vmin=-50, vmax=50) 
    axes[1].set_title("Preprocessed data with original diffusion data as overlay")
    axes[1].axis("off")
    plt.show()